<a href="https://colab.research.google.com/github/M-torki/ECG-Classification/blob/main/Sprint4_BestChallenge_icbeb_CatBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is based on the paper: 

**[Deep Learning for ECG Analysis: Benchmarks and Insights from PTB-XL](https://ieeexplore.ieee.org/document/9190034)**

The related codes are available on this [Github](https://github.com/helme/ecg_ptbxl_benchmarking/) 

link to [PTB XL](https://physionet.org/content/ptb-xl/1.0.1/) database, 
link to [ICBEB2018](http://2018.icbeb.org/Challenge.html) database

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
cd /gdrive/MyDrive/

/gdrive/MyDrive


In [ ]:
!pip install wfdb

     |████████████████████████████████| 137 kB 7.8 MB/s 


In [ ]:
# !git clone https://github.com/helme/ecg_ptbxl_benchmarking/

In [ ]:
cd ./ecg_ptbxl_benchmarking/

/gdrive/.shortcut-targets-by-id/1j3ncHY23bba4nXCRAt52Fr8YgGZpml9-/ecg_ptbxl_benchmarking


In [ ]:
cd code/

/gdrive/.shortcut-targets-by-id/1j3ncHY23bba4nXCRAt52Fr8YgGZpml9-/ecg_ptbxl_benchmarking/code


In [ ]:
#@title utils
import os
import sys
import re
import glob
import pickle
import copy

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import wfdb
import ast
from sklearn.metrics import classification_report, fbeta_score, roc_auc_score, roc_curve, roc_curve, auc
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer
from matplotlib.axes._axes import _log as matplotlib_axes_logger
import warnings

# EVALUATION STUFF
def generate_results(idxs, y_true, y_pred, thresholds):
    return evaluate_experiment(y_true[idxs], y_pred[idxs], thresholds)

def evaluate_experiment(y_true, y_pred, thresholds=None):
    results = {}

    if not thresholds is None:
        # binary predictions
        y_pred_binary = apply_thresholds(y_pred, thresholds)
        # PhysioNet/CinC Challenges metrics
        challenge_scores = challenge_metrics(y_true, y_pred_binary, beta1=2, beta2=2)
        results['F_beta_macro'] = challenge_scores['F_beta_macro']
        results['G_beta_macro'] = challenge_scores['G_beta_macro']

    # label based metric
    results['macro_auc'] = roc_auc_score(y_true, y_pred, average='macro')
    
    df_result = pd.DataFrame(results, index=[0])
    return df_result

def challenge_metrics(y_true, y_pred, beta1=2, beta2=2, class_weights=None, single=False):
    f_beta = 0
    g_beta = 0
    if single: # if evaluating single class in case of threshold-optimization
        sample_weights = np.ones(y_true.sum(axis=1).shape)
    else:
        sample_weights = y_true.sum(axis=1)
    for classi in range(y_true.shape[1]):
        y_truei, y_predi = y_true[:,classi], y_pred[:,classi]
        TP, FP, TN, FN = 0.,0.,0.,0.
        for i in range(len(y_predi)):
            sample_weight = sample_weights[i]
            if y_truei[i]==y_predi[i]==1: 
                TP += 1./sample_weight
            if ((y_predi[i]==1) and (y_truei[i]!=y_predi[i])): 
                FP += 1./sample_weight
            if y_truei[i]==y_predi[i]==0: 
                TN += 1./sample_weight
            if ((y_predi[i]==0) and (y_truei[i]!=y_predi[i])): 
                FN += 1./sample_weight 
        f_beta_i = ((1+beta1**2)*TP)/((1+beta1**2)*TP + FP + (beta1**2)*FN)
        g_beta_i = (TP)/(TP+FP+beta2*FN)

        f_beta += f_beta_i
        g_beta += g_beta_i

    return {'F_beta_macro':f_beta/y_true.shape[1], 'G_beta_macro':g_beta/y_true.shape[1]}

def get_appropriate_bootstrap_samples(y_true, n_bootstraping_samples):
    samples=[]
    while True:
        ridxs = np.random.randint(0, len(y_true), len(y_true))
        if y_true[ridxs].sum(axis=0).min() != 0:
            samples.append(ridxs)
            if len(samples) == n_bootstraping_samples:
                break
    return samples

def find_optimal_cutoff_threshold(target, predicted):
    """ 
    Find the optimal probability cutoff point for a classification model related to event rate
    """
    fpr, tpr, threshold = roc_curve(target, predicted)
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = threshold[optimal_idx]
    return optimal_threshold

def find_optimal_cutoff_thresholds(y_true, y_pred):
	return [find_optimal_cutoff_threshold(y_true[:,i], y_pred[:,i]) for i in range(y_true.shape[1])]

def find_optimal_cutoff_threshold_for_Gbeta(target, predicted, n_thresholds=100):
    thresholds = np.linspace(0.00,1,n_thresholds)
    scores = [challenge_metrics(target, predicted>t, single=True)['G_beta_macro'] for t in thresholds]
    optimal_idx = np.argmax(scores)
    return thresholds[optimal_idx]

def find_optimal_cutoff_thresholds_for_Gbeta(y_true, y_pred):
    print("optimize thresholds with respect to G_beta")
    return [find_optimal_cutoff_threshold_for_Gbeta(y_true[:,k][:,np.newaxis], y_pred[:,k][:,np.newaxis]) for k in tqdm(range(y_true.shape[1]))]

def apply_thresholds(preds, thresholds):
	"""
		apply class-wise thresholds to prediction score in order to get binary format.
		BUT: if no score is above threshold, pick maximum. This is needed due to metric issues.
	"""
	tmp = []
	for p in preds:
		tmp_p = (p > thresholds).astype(int)
		if np.sum(tmp_p) == 0:
			tmp_p[np.argmax(p)] = 1
		tmp.append(tmp_p)
	tmp = np.array(tmp)
	return tmp

# DATA PROCESSING STUFF

def load_dataset(path, sampling_rate, release=False):
    if path.split('/')[-2] == 'ptbxl':
        # load and convert annotation data
        Y = pd.read_csv(path+'ptbxl_database.csv', index_col='ecg_id')
        Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

        # Load raw signal data
        X = load_raw_data_ptbxl(Y, sampling_rate, path)

    elif path.split('/')[-2] == 'ICBEB':
        # load and convert annotation data
        Y = pd.read_csv(path+'icbeb_database.csv', index_col='ecg_id')
        Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

        # Load raw signal data
        X = load_raw_data_icbeb(Y, sampling_rate, path)

    return X, Y


def load_raw_data_icbeb(df, sampling_rate, path):

    if sampling_rate == 100:
        if os.path.exists(path + 'raw100.npy'):
            data = np.load(path+'raw100.npy', allow_pickle=True)
        else:
            data = [wfdb.rdsamp(path + 'records100/'+str(f)) for f in tqdm(df.index)]
            data = np.array([signal for signal, meta in data])
            pickle.dump(data, open(path+'raw100.npy', 'wb'), protocol=4)
    elif sampling_rate == 500:
        if os.path.exists(path + 'raw500.npy'):
            data = np.load(path+'raw500.npy', allow_pickle=True)
        else:
            data = [wfdb.rdsamp(path + 'records500/'+str(f)) for f in tqdm(df.index)]
            data = np.array([signal for signal, meta in data])
            pickle.dump(data, open(path+'raw500.npy', 'wb'), protocol=4)
    return data

def load_raw_data_ptbxl(df, sampling_rate, path):
    if sampling_rate == 100:
        if os.path.exists(path + 'raw100.npy'):
            data = np.load(path+'raw100.npy', allow_pickle=True)
        else:
            data = [wfdb.rdsamp(path+f) for f in tqdm(df.filename_lr)]
            data = np.array([signal for signal, meta in data])
            pickle.dump(data, open(path+'raw100.npy', 'wb'), protocol=4)
    elif sampling_rate == 500:
        if os.path.exists(path + 'raw500.npy'):
            data = np.load(path+'raw500.npy', allow_pickle=True)
        else:
            data = [wfdb.rdsamp(path+f) for f in tqdm(df.filename_hr)]
            data = np.array([signal for signal, meta in data])
            pickle.dump(data, open(path+'raw500.npy', 'wb'), protocol=4)
    return data

def compute_label_aggregations(df, folder, ctype):

    df['scp_codes_len'] = df.scp_codes.apply(lambda x: len(x))

    aggregation_df = pd.read_csv(folder+'scp_statements.csv', index_col=0)

    if ctype in ['diagnostic', 'subdiagnostic', 'superdiagnostic']:

        def aggregate_all_diagnostic(y_dic):
            tmp = []
            for key in y_dic.keys():
                if key in diag_agg_df.index:
                    tmp.append(key)
            return list(set(tmp))

        def aggregate_subdiagnostic(y_dic):
            tmp = []
            for key in y_dic.keys():
                if key in diag_agg_df.index:
                    c = diag_agg_df.loc[key].diagnostic_subclass
                    if str(c) != 'nan':
                        tmp.append(c)
            return list(set(tmp))

        def aggregate_diagnostic(y_dic):
            tmp = []
            for key in y_dic.keys():
                if key in diag_agg_df.index:
                    c = diag_agg_df.loc[key].diagnostic_class
                    if str(c) != 'nan':
                        tmp.append(c)
            return list(set(tmp))

        diag_agg_df = aggregation_df[aggregation_df.diagnostic == 1.0]
        if ctype == 'diagnostic':
            df['diagnostic'] = df.scp_codes.apply(aggregate_all_diagnostic)
            df['diagnostic_len'] = df.diagnostic.apply(lambda x: len(x))
        elif ctype == 'subdiagnostic':
            df['subdiagnostic'] = df.scp_codes.apply(aggregate_subdiagnostic)
            df['subdiagnostic_len'] = df.subdiagnostic.apply(lambda x: len(x))
        elif ctype == 'superdiagnostic':
            df['superdiagnostic'] = df.scp_codes.apply(aggregate_diagnostic)
            df['superdiagnostic_len'] = df.superdiagnostic.apply(lambda x: len(x))
    elif ctype == 'form':
        form_agg_df = aggregation_df[aggregation_df.form == 1.0]

        def aggregate_form(y_dic):
            tmp = []
            for key in y_dic.keys():
                if key in form_agg_df.index:
                    c = key
                    if str(c) != 'nan':
                        tmp.append(c)
            return list(set(tmp))

        df['form'] = df.scp_codes.apply(aggregate_form)
        df['form_len'] = df.form.apply(lambda x: len(x))
    elif ctype == 'rhythm':
        rhythm_agg_df = aggregation_df[aggregation_df.rhythm == 1.0]

        def aggregate_rhythm(y_dic):
            tmp = []
            for key in y_dic.keys():
                if key in rhythm_agg_df.index:
                    c = key
                    if str(c) != 'nan':
                        tmp.append(c)
            return list(set(tmp))

        df['rhythm'] = df.scp_codes.apply(aggregate_rhythm)
        df['rhythm_len'] = df.rhythm.apply(lambda x: len(x))
    elif ctype == 'all':
        df['all_scp'] = df.scp_codes.apply(lambda x: list(set(x.keys())))

    return df

def select_data(XX,YY, ctype, min_samples, outputfolder):
    # convert multilabel to multi-hot
    mlb = MultiLabelBinarizer()

    if ctype == 'diagnostic':
        X = XX[YY.diagnostic_len > 0]
        Y = YY[YY.diagnostic_len > 0]
        mlb.fit(Y.diagnostic.values)
        y = mlb.transform(Y.diagnostic.values)
    elif ctype == 'subdiagnostic':
        counts = pd.Series(np.concatenate(YY.subdiagnostic.values)).value_counts()
        counts = counts[counts > min_samples]
        YY.subdiagnostic = YY.subdiagnostic.apply(lambda x: list(set(x).intersection(set(counts.index.values))))
        YY['subdiagnostic_len'] = YY.subdiagnostic.apply(lambda x: len(x))
        X = XX[YY.subdiagnostic_len > 0]
        Y = YY[YY.subdiagnostic_len > 0]
        mlb.fit(Y.subdiagnostic.values)
        y = mlb.transform(Y.subdiagnostic.values)
    elif ctype == 'superdiagnostic':
        counts = pd.Series(np.concatenate(YY.superdiagnostic.values)).value_counts()
        counts = counts[counts > min_samples]
        YY.superdiagnostic = YY.superdiagnostic.apply(lambda x: list(set(x).intersection(set(counts.index.values))))
        YY['superdiagnostic_len'] = YY.superdiagnostic.apply(lambda x: len(x))
        X = XX[YY.superdiagnostic_len > 0]
        Y = YY[YY.superdiagnostic_len > 0]
        mlb.fit(Y.superdiagnostic.values)
        y = mlb.transform(Y.superdiagnostic.values)
    elif ctype == 'form':
        # filter
        counts = pd.Series(np.concatenate(YY.form.values)).value_counts()
        counts = counts[counts > min_samples]
        YY.form = YY.form.apply(lambda x: list(set(x).intersection(set(counts.index.values))))
        YY['form_len'] = YY.form.apply(lambda x: len(x))
        # select
        X = XX[YY.form_len > 0]
        Y = YY[YY.form_len > 0]
        mlb.fit(Y.form.values)
        y = mlb.transform(Y.form.values)
    elif ctype == 'rhythm':
        # filter 
        counts = pd.Series(np.concatenate(YY.rhythm.values)).value_counts()
        counts = counts[counts > min_samples]
        YY.rhythm = YY.rhythm.apply(lambda x: list(set(x).intersection(set(counts.index.values))))
        YY['rhythm_len'] = YY.rhythm.apply(lambda x: len(x))
        # select
        X = XX[YY.rhythm_len > 0]
        Y = YY[YY.rhythm_len > 0]
        mlb.fit(Y.rhythm.values)
        y = mlb.transform(Y.rhythm.values)
    elif ctype == 'all':
        # filter 
        counts = pd.Series(np.concatenate(YY.all_scp.values)).value_counts()
        counts = counts[counts > min_samples]
        YY.all_scp = YY.all_scp.apply(lambda x: list(set(x).intersection(set(counts.index.values))))
        YY['all_scp_len'] = YY.all_scp.apply(lambda x: len(x))
        # select
        X = XX[YY.all_scp_len > 0]
        Y = YY[YY.all_scp_len > 0]
        mlb.fit(Y.all_scp.values)
        y = mlb.transform(Y.all_scp.values)
    else:
        pass

    # save LabelBinarizer
    with open(outputfolder+'mlb.pkl', 'wb') as tokenizer:
        pickle.dump(mlb, tokenizer)

    return X, Y, y, mlb

def preprocess_signals(X_train, X_validation, X_test, outputfolder):
    # Standardize data such that mean 0 and variance 1
    ss = StandardScaler()
    ss.fit(np.vstack(X_train).flatten()[:,np.newaxis].astype(float))
    
    # Save Standardizer data
    with open(outputfolder+'standard_scaler.pkl', 'wb') as ss_file:
        pickle.dump(ss, ss_file)

    return apply_standardizer(X_train, ss), apply_standardizer(X_validation, ss), apply_standardizer(X_test, ss)

def apply_standardizer(X, ss):
    X_tmp = []
    for x in X:
        x_shape = x.shape
        X_tmp.append(ss.transform(x.flatten()[:,np.newaxis]).reshape(x_shape))
    X_tmp = np.array(X_tmp)
    return X_tmp


# DOCUMENTATION STUFF

def generate_ptbxl_summary_table(selection=None, folder='../output/'):

    exps = ['exp0', 'exp1', 'exp1.1', 'exp1.1.1', 'exp2', 'exp3']
    metric1 = 'macro_auc' 

    # get models
    models = {}
    for i, exp in enumerate(exps):
        if selection is None:
            exp_models = [m.split('/')[-1] for m in glob.glob(folder+str(exp)+'/models/*')]
        else:
            exp_models = selection
        if i == 0:
            models = set(exp_models)
        else:
            models = models.union(set(exp_models))

    results_dic = {'Method':[], 
                'exp0_AUC':[], 
                'exp1_AUC':[], 
                'exp1.1_AUC':[], 
                'exp1.1.1_AUC':[], 
                'exp2_AUC':[],
                'exp3_AUC':[]
                }

    for m in models:
        results_dic['Method'].append(m)
        
        for e in exps:
            
            try:
                me_res = pd.read_csv(folder+str(e)+'/models/'+str(m)+'/results/te_results.csv', index_col=0)
    
                mean1 = me_res.loc['point'][metric1]
                unc1 = max(me_res.loc['upper'][metric1]-me_res.loc['point'][metric1], me_res.loc['point'][metric1]-me_res.loc['lower'][metric1])

                results_dic[e+'_AUC'].append("%.3f(%.2d)" %(np.round(mean1,3), int(unc1*1000)))

            except FileNotFoundError:
                results_dic[e+'_AUC'].append("--")
            
            
    df = pd.DataFrame(results_dic)
    df_index = df[df.Method.isin(['naive', 'ensemble'])]
    df_rest = df[~df.Method.isin(['naive', 'ensemble'])]
    df = pd.concat([df_rest, df_index])
    df.to_csv(folder+'results_ptbxl.csv')

    titles = [
        '### 1. PTB-XL: all statements',
        '### 2. PTB-XL: diagnostic statements',
        '### 3. PTB-XL: Diagnostic subclasses',
        '### 4. PTB-XL: Diagnostic superclasses',
        '### 5. PTB-XL: Form statements',
        '### 6. PTB-XL: Rhythm statements'        
    ]

    # helper output function for markdown tables
    our_work = 'https://arxiv.org/abs/2004.13701'
    our_repo = 'https://github.com/helme/ecg_ptbxl_benchmarking/'
    md_source = ''
    for i, e in enumerate(exps):
        md_source += '\n '+titles[i]+' \n \n'
        md_source += '| Model | AUC &darr; | paper/source | code | \n'
        md_source += '|---:|:---|:---|:---| \n'
        for row in df_rest[['Method', e+'_AUC']].sort_values(e+'_AUC', ascending=False).values:
            md_source += '| ' + row[0].replace('fastai_', '') + ' | ' + row[1] + ' | [our work]('+our_work+') | [this repo]('+our_repo+')| \n'
    print(md_source)

def ICBEBE_table(selection=None, folder='../output/'):
    cols = ['macro_auc', 'F_beta_macro', 'G_beta_macro']

    if selection is None:
        models = [m.split('/')[-1].split('_pretrained')[0] for m in glob.glob(folder+'exp_ICBEB/models/*')]
    else:
        models = [] 
        for s in selection:
            #if s != 'Wavelet+NN':
                models.append(s)

    data = []
    for model in models:
        me_res = pd.read_csv(folder+'exp_ICBEB/models/'+model+'/results/te_results.csv', index_col=0)
        mcol=[]
        for col in cols:
            mean = me_res.ix['point'][col]
            unc = max(me_res.ix['upper'][col]-me_res.ix['point'][col], me_res.ix['point'][col]-me_res.ix['lower'][col])
            mcol.append("%.3f(%.2d)" %(np.round(mean,3), int(unc*1000)))
        data.append(mcol)
    data = np.array(data)

    df = pd.DataFrame(data, columns=cols, index=models)
    df.to_csv(folder+'results_icbeb.csv')

    df_rest = df[~df.index.isin(['naive', 'ensemble'])]
    df_rest = df_rest.sort_values('macro_auc', ascending=False)
    our_work = 'https://arxiv.org/abs/2004.13701'
    our_repo = 'https://github.com/helme/ecg_ptbxl_benchmarking/'

    md_source = '| Model | AUC &darr; |  F_beta=2 | G_beta=2 | paper/source | code | \n'
    md_source += '|---:|:---|:---|:---|:---|:---| \n'
    for i, row in enumerate(df_rest[cols].values):
        md_source += '| ' + df_rest.index[i].replace('fastai_', '') + ' | ' + row[0] + ' | ' + row[1] + ' | ' + row[2] + ' | [our work]('+our_work+') | [this repo]('+our_repo+')| \n'
    print(md_source)
    

In [ ]:
import glob
import random
import os
import argparse
import scipy.io as sio
from keras import backend as K
from sklearn.model_selection import train_test_split
import csv
import numpy
import numpy as np

import pandas as pd
import tensorflow as tf
import scipy
from tensorflow.python.client import device_lib
import keras
from keras.models import Sequential, load_model
from keras.layers import LSTM, GRU, TimeDistributed, Bidirectional, LeakyReLU
from keras.layers import Dense, Dropout, Activation, Flatten,  Input, Reshape, GRU, CuDNNGRU
from keras.layers import Convolution1D, MaxPool1D, GlobalAveragePooling1D,concatenate,AveragePooling1D
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping
from keras.models import Model
from keras import initializers, regularizers, constraints
from keras.layers import Layer
import numpy as np
from keras.layers.normalization import BatchNormalization
from keras import regularizers
import scipy.io as sio
from os import listdir
from keras.optimizers import Adam

#sampling frequency=100




In [ ]:
from utils import utils

sampling_frequency=100
datafolder= '/gdrive/My Drive/ICBEB/'
task='all'
outputfolder='../output0/'

# Load data
data, raw_labels = utils.load_dataset(datafolder, sampling_frequency)
# Preprocess label data
labels = utils.compute_label_aggregations(raw_labels, datafolder, task)
# Select relevant data and convert to one-hot
data, labels, Y, _ = utils.select_data(data, labels, task, min_samples=0, outputfolder=outputfolder)

# 1-9 for training 
X_train = data[labels.strat_fold < 10]
y_train = Y[labels.strat_fold < 10]
# 10 for validation
X_val = data[labels.strat_fold == 10]
y_val = Y[labels.strat_fold == 10]

num_classes = 9         # <=== number of classes in the finetuning dataset
input_shape = [1000,12] # <=== shape of samples, [None, 12] in case of different lengths

X_train.shape, y_train.shape, X_val.shape, y_val.shape

((6187,), (6187, 9), (690,), (690, 9))

In [ ]:
X_train[0].shape

(1500, 12)

In [ ]:
X_tr = []
for i in range(len(X_train)):
    x = []
    for j in range(12):
        p = X_train[i][:1000,j]
        if p.shape[0]!= 1000:
            d = abs(p.shape[0]-1000)//2
            p = np.pad(p,(d,d))
        x.append(p)
    X_tr.append(np.transpose(x))
X_tr = np.array(X_tr)
X_tr.shape

(6187, 1000, 12)

In [ ]:
X_te = []
for i in range(len(X_val)):
    x = []
    for j in range(12):
        p = X_val[i][:1000,j]
        if p.shape[0]!= 1000:
            d = abs(p.shape[0]-1000)//2
            p = np.pad(p,(d,d))
        x.append(p)
    X_te.append(np.transpose(x))
X_te = np.array(X_te)
X_te.shape

(690, 1000, 12)

In [ ]:
def dot_product(x, kernel):
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)

class AttentionWithContext(Layer):
    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs): 
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform') 
        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer) 
        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint) 
        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)
 
    def build(self, input_shape):
        assert len(input_shape) == 3
        self.W = self.add_weight(shape=(input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight(shape=(input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint) 
            self.u = self.add_weight(shape=(input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint) 
        super(AttentionWithContext, self).build(input_shape)
 
    def compute_mask(self, input, input_mask=None):
        return None
 
    def call(self, x, mask=None):
        uit = dot_product(x, self.W) 
        if self.bias:
            uit += self.b 
        uit = K.tanh(uit)
        ait = dot_product(uit, self.u) 
        a = K.exp(ait)
        if mask is not None:
            a *= K.cast(mask, K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx()) 
        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)
 
    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]


In [ ]:
main_input = Input(shape=(1000,12), dtype='float32', name='main_input')
x = Convolution1D(12, 3, padding='same')(main_input)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 3, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 24, strides = 2, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Dropout(0.2)(x)
x = Convolution1D(12, 3, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 3, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 24, strides = 2, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Dropout(0.2)(x)
x = Convolution1D(12, 3, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 3, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 24, strides = 2, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Dropout(0.2)(x)
x = Convolution1D(12, 3, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 3, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 24, strides = 2, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Dropout(0.2)(x)
x = Convolution1D(12, 3, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 3, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 48, strides = 2, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
cnnout = Dropout(0.2)(x)
x = Bidirectional(CuDNNGRU(12, input_shape=(32,12),return_sequences=True,return_state=False))(cnnout)
x = LeakyReLU(alpha=0.3)(x)
x = Dropout(0.2)(x)
x = AttentionWithContext()(x)
x = BatchNormalization()(x)
last_dense = LeakyReLU(alpha=0.3)(x)
x = Dropout(0.2)(last_dense)
main_output = Dense(num_classes,activation='sigmoid')(x)

In [ ]:
model = Model(main_input,main_output)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      [(None, 1000, 12)]        0         
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 1000, 12)          444       
_________________________________________________________________
leaky_re_lu_17 (LeakyReLU)   (None, 1000, 12)          0         
_________________________________________________________________
conv1d_16 (Conv1D)           (None, 1000, 12)          444       
_________________________________________________________________
leaky_re_lu_18 (LeakyReLU)   (None, 1000, 12)          0         
_________________________________________________________________
conv1d_17 (Conv1D)           (None, 500, 12)           3468      
_________________________________________________________________
leaky_re_lu_19 (LeakyReLU)   (None, 500, 12)           0   

In [ ]:
intermediate_layer_model = Model(main_input,last_dense)

intermediate_layer_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      [(None, 1000, 12)]        0         
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 1000, 12)          444       
_________________________________________________________________
leaky_re_lu_17 (LeakyReLU)   (None, 1000, 12)          0         
_________________________________________________________________
conv1d_16 (Conv1D)           (None, 1000, 12)          444       
_________________________________________________________________
leaky_re_lu_18 (LeakyReLU)   (None, 1000, 12)          0         
_________________________________________________________________
conv1d_17 (Conv1D)           (None, 500, 12)           3468      
_________________________________________________________________
leaky_re_lu_19 (LeakyReLU)   (None, 500, 12)           0   

In [ ]:
model.compile(optimizer=Adam(learning_rate=1e-4),
                loss='binary_crossentropy',
                metrics=['AUC'])

In [ ]:
model.fit(X_tr, y_train, 
          validation_split=0.1,
          epochs=20,
          batch_size=128,
          shuffle=True)

Epoch 1/20
44/44 [==============================] - 4s 24ms/step - loss: 0.1182 - auc: 0.9698 - val_loss: 0.1325 - val_auc: 0.9615
Epoch 2/20
44/44 [==============================] - 1s 20ms/step - loss: 0.1168 - auc: 0.9701 - val_loss: 0.1302 - val_auc: 0.9630
Epoch 3/20
44/44 [==============================] - 1s 20ms/step - loss: 0.1166 - auc: 0.9708 - val_loss: 0.1297 - val_auc: 0.9624
Epoch 4/20
44/44 [==============================] - 1s 20ms/step - loss: 0.1175 - auc: 0.9695 - val_loss: 0.1282 - val_auc: 0.9637
Epoch 5/20
44/44 [==============================] - 1s 20ms/step - loss: 0.1182 - auc: 0.9697 - val_loss: 0.1297 - val_auc: 0.9629
Epoch 6/20
44/44 [==============================] - 1s 20ms/step - loss: 0.1170 - auc: 0.9707 - val_loss: 0.1306 - val_auc: 0.9627
Epoch 7/20
44/44 [==============================] - 1s 20ms/step - loss: 0.1176 - auc: 0.9704 - val_loss: 0.1329 - val_auc: 0.9620
Epoch 8/20
44/44 [==============================] - 1s 20ms/step - loss: 0.1165 - a

In [ ]:
y_pred = model.predict(X_te)
utils.evaluate_experiment(y_val, y_pred)

,macro_auc
0,0.945772


In [ ]:
from sklearn.metrics import classification_report, fbeta_score, roc_auc_score, roc_curve, roc_curve, auc

def find_optimal_cutoff_threshold(target, predicted):
    """ 
    Find the optimal probability cutoff point for a classification model related to event rate
    """
    fpr, tpr, threshold = roc_curve(target, predicted)
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = threshold[optimal_idx]
    return optimal_threshold

def find_optimal_cutoff_thresholds(y_true, y_pred):
	return [find_optimal_cutoff_threshold(y_true[:,i], y_pred[:,i]) for i in range(y_true.shape[1])]

thresholds =  find_optimal_cutoff_thresholds_for_Gbeta(y_val , y_pred)


  0%|          | 0/9 [00:00<?, ?it/s]

optimize thresholds with respect to G_beta


100%|██████████| 9/9 [00:07<00:00,  1.28it/s]


In [ ]:
y_pred_binary = apply_thresholds(y_pred, thresholds)
y_pred_binary[0]

array([0, 1, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
utils.evaluate_experiment(y_val, y_pred_binary)

,macro_auc
0,0.886316


In [ ]:
from sklearn.metrics import (confusion_matrix,
                             precision_score, recall_score, f1_score,
                             precision_recall_curve, average_precision_score , balanced_accuracy_score)
diagnosis = ['NORM', 'AFIB', '1AVB', 'CLBBB', 'CRBBB', 'PAC', 'VPC', 'STD_', 'STE_']
def specificity_score(y_true, y_pred):
    m = confusion_matrix(y_true, y_pred, labels=[0, 1])
    spc = m[0, 0] * 1.0 / (m[0, 0] + m[0, 1])
    return spc

def get_scores(y_true, y_pred, score_fun):
    nclasses = np.shape(y_true)[1]
    scores = []
    for name, fun in score_fun.items():
        scores += [[fun(y_true[:, k], y_pred[:, k]) for k in range(nclasses)]]
    return np.array(scores).T


score_fun = {'Precision': precision_score,
             'Recall': recall_score, 'Specificity': specificity_score,
             'F1 score': f1_score , 'balanced accuracy': balanced_accuracy_score}

scores_list = []
for y_pred in [y_pred_binary]:
    # Compute scores
    scores = get_scores(y_val, y_pred, score_fun)
    # Put them into a data frame
    scores_df = pd.DataFrame(scores,index=diagnosis, columns=score_fun.keys())
    # Append
    scores_list.append(scores_df)
# Concatenate dataframes
scores_all_df = pd.concat(scores_list, axis=1, keys=[''])
# Change multiindex levels
scores_all_df = scores_all_df.swaplevel(0, 1, axis=1)
scores_all_df = scores_all_df.reindex(level=0, columns=score_fun.keys())

In [ ]:
#Gbeta thershold
scores_all_df

,Precision,Recall,Specificity,F1 score,balanced accuracy
,,,,,
NORM,0.846154,0.916667,0.980583,0.880000,0.948625
AFIB,0.896825,0.918699,0.977072,0.907631,0.947886
1AVB,0.863636,0.791667,0.995495,0.826087,0.893581
CLBBB,0.901478,0.948187,0.959759,0.924242,0.953973
CRBBB,0.754902,0.836957,0.958194,0.793814,0.897575
PAC,0.686275,0.573770,0.974563,0.625000,0.774167
VPC,0.750000,0.862069,0.958541,0.802139,0.910305
STD_,0.666667,0.636364,0.989521,0.651163,0.812942
STE_,0.675676,0.714286,0.961290,0.694444,0.837788


In [ ]:
# %% Confusion matrices (Supplementary Table 1)
from sklearn.metrics import (confusion_matrix,
                             precision_score, recall_score, f1_score,
                             precision_recall_curve, average_precision_score)

import xarray as xr

M = [[confusion_matrix(y_val[:, k], y_pred[:, k], labels=[0, 1])
      for k in range(9)] for y_pred in [y_pred_binary]]

M_xarray = xr.DataArray(np.array(M),
                        dims=['predictor', 'diagnosis', 'true label', 'predicted label'],
                        coords={'predictor': ['CNN+RNN'],
                                'diagnosis': diagnosis,
                                'true label': ['not present', 'present'],
                                'predicted label': ['not present', 'present']})
confusion_matrices = M_xarray.to_dataframe('n')
confusion_matrices = confusion_matrices.reorder_levels([1, 2, 3, 0], axis=0)
confusion_matrices = confusion_matrices.unstack()
confusion_matrices = confusion_matrices.unstack()
confusion_matrices = confusion_matrices['n']
confusion_matrices

predictor                 CNN+RNN        
predicted label       not present present
diagnosis true label                     
NORM      not present         606      12
          present               6      66
AFIB      not present         554      13
          present              10     113
1AVB      not present         663       3
          present               5      19
CLBBB     not present         477      20
          present              10     183
CRBBB     not present         573      25
          present              15      77
PAC       not present         613      16
          present              26      35
VPC       not present         578      25
          present              12      75
STD_      not present         661       7
          present               8      14
STE_      not present         596      24
          present              20      50

In [ ]:
intermediate_output = intermediate_layer_model.predict(X_tr) 
intermediate_output = pd.DataFrame(data=intermediate_output)
intermediate_output

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,3.001643,-0.067764,-0.288871,-0.090582,2.411433,1.748587,-0.402312,0.430912,-0.392957,-0.244924,0.781219,1.680753,-0.359231,2.324959,-0.270377,1.821432,2.283545,-0.327521,-0.127641,-0.233211,-0.170167,2.150958,1.642843,2.255541
1,0.944204,2.203956,2.494365,-0.193275,-0.153300,1.267270,1.886377,-0.088683,1.735864,1.757166,2.951988,-0.095238,-0.255146,2.853224,-0.059378,-0.341435,-0.354121,3.077388,-0.179307,0.016734,-0.251803,-0.092571,-0.228424,-0.242189
2,-0.219171,-0.185172,-0.272619,-0.056880,-0.170273,-0.422777,1.851090,-0.087204,1.447175,-0.321971,-0.343484,2.818882,-0.302385,-0.214753,3.071851,1.483443,-0.293914,-0.011953,3.601426,1.099658,3.204190,-0.212399,0.917357,1.635759
3,-0.172477,-0.198357,2.611312,2.086379,-0.173512,1.203426,1.072673,-0.200906,0.458440,1.297238,0.528037,-0.090203,1.569444,-0.074765,0.734828,1.426463,-0.312277,-0.086207,-0.094051,-0.063027,-0.084447,0.493436,0.631761,0.070878
4,3.224120,-0.010500,-0.257060,-0.105387,1.687323,2.732973,-0.403518,0.308186,-0.388403,-0.154453,1.839732,1.963377,-0.375995,2.495319,-0.280214,1.866578,2.273375,-0.305717,-0.185327,-0.244148,-0.233412,2.302249,1.115411,2.182754
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6182,0.523405,1.813215,-0.289492,-0.043818,2.340290,-0.275741,-0.310848,2.998348,-0.023842,0.220957,-0.072603,-0.037156,-0.012081,2.134110,-0.032463,-0.057847,0.369818,-0.115334,0.503904,-0.070384,-0.095565,0.484909,2.325752,-0.109211
6183,3.402399,1.001953,2.576252,0.859229,0.017726,-0.262102,1.462629,-0.101996,-0.241797,1.657388,-0.087862,-0.189367,-0.046283,-0.186677,0.250819,-0.377086,2.195347,1.497748,2.627158,2.663933,0.268404,-0.083794,-0.335468,0.283352
6184,-0.314032,-0.133376,-0.437824,1.919265,-0.353851,2.770180,-0.164570,4.375890,1.988236,-0.193792,-0.136241,3.281760,-0.132586,-0.291921,1.406449,-0.193715,1.293892,-0.046822,-0.213633,4.084955,-0.027310,-0.511739,-0.513955,-0.316522
6185,-0.213868,-0.202970,-0.385915,-0.144116,-0.066047,-0.479793,1.624040,-0.035350,1.320960,-0.356047,-0.365784,3.151830,-0.308871,-0.157329,2.828130,1.744192,-0.414381,-0.137230,3.320789,0.798270,3.055030,-0.157947,2.769519,2.179981


In [ ]:
intermediate_output.shape

(6187, 24)

In [ ]:
# submission_cnn = incep.predict(X_val)
intermediate_test_output = intermediate_layer_model.predict(X_te)
intermediate_test_output = pd.DataFrame(data=intermediate_test_output)
intermediate_test_output

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,-0.255392,-0.211745,-0.406428,-0.142480,-0.125017,-0.489880,1.848378,0.334954,1.612910,-0.359512,-0.393796,3.109035,-0.252718,-0.182750,3.527593,1.619462,-0.462926,-0.075328,3.171779,0.617038,3.235300,-0.294856,2.669544,1.866967
1,0.525714,0.062044,1.557777,0.059896,-0.107384,-0.300601,1.515254,-0.036957,0.816227,1.588835,-0.238807,-0.103106,-0.082493,-0.138587,0.712973,-0.222239,0.889631,0.847310,1.873084,1.778764,0.631984,0.163307,-0.215947,0.368411
2,1.285223,2.865669,2.576134,-0.257837,-0.213428,2.624957,1.912687,-0.128523,1.846269,1.780268,4.874685,-0.073190,-0.312853,2.891600,-0.153347,-0.203334,-0.399434,2.994854,-0.207030,-0.014133,-0.287892,-0.058565,-0.339682,-0.276051
3,-0.273353,0.120967,-0.437371,1.034279,-0.311853,2.482226,-0.168039,3.370830,1.743863,-0.144708,0.694945,2.071692,-0.119326,-0.290215,1.416375,0.140978,2.513539,-0.045433,-0.206044,3.490529,-0.226120,-0.478638,-0.496032,-0.333505
4,0.369120,3.283067,-0.244850,-0.201509,-0.143654,2.830542,0.478915,0.732054,1.763713,1.649326,2.323838,-0.063304,-0.283985,1.099096,-0.151460,1.009237,-0.247494,1.205474,-0.204992,0.569872,-0.281708,0.346507,-0.329852,-0.255727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,2.074163,-0.020272,-0.282366,-0.064773,1.562740,2.176630,-0.400651,0.538170,-0.397890,-0.269159,1.730328,1.151343,-0.287263,2.106034,-0.280524,1.900683,1.505880,-0.343726,-0.165426,-0.243654,-0.246007,3.098737,1.659626,2.171075
686,3.849508,1.430745,2.997154,1.051703,-0.030274,-0.278380,1.630284,-0.172496,-0.230444,1.733097,-0.098901,-0.208783,-0.040841,-0.207346,0.692708,-0.428068,2.437585,2.315858,2.813515,3.012163,0.089091,-0.162041,-0.395679,-0.040044
687,1.831842,-0.037180,-0.316673,-0.055374,1.922740,1.058200,-0.399168,0.897470,-0.379024,-0.285368,0.560102,1.390820,-0.314774,1.551067,-0.247142,1.790096,1.873255,-0.309355,-0.084726,-0.191027,-0.127434,1.644878,2.002734,2.035933
688,1.418620,1.312839,0.890478,-0.092359,-0.009147,1.888826,-0.027134,0.092668,0.727996,1.634679,1.880245,0.228764,-0.288326,1.956409,-0.159836,0.690284,0.083759,0.071297,-0.164456,-0.050125,-0.197531,0.608387,-0.101953,0.002561


In [ ]:
import xgboost as xgb
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score , auc

# create XGBoost instance with default hyper-parameters
# xgb_estimator = xgb.XGBClassifier(objective='binary:logistic' , max_depth=3 , n_estimators=100 ,colsample_bytree=0.7 , )
xgb_estimator = xgb.XGBClassifier(objective='binary:logistic' , max_depth=15 , n_estimators=200  , learning_rate=0.01, subsample=0.7
                                  , gamma=0.8 , )

# create MultiOutputClassifier instance with XGBoost model inside
multilabel_model = MultiOutputClassifier(xgb_estimator)

# fit the model
multilabel_model.fit(intermediate_output, y_train)

# evaluate on test data
print('Accuracy on test data: {:.1f}%'.format(accuracy_score(y_val, multilabel_model.predict(intermediate_test_output))*100))

Accuracy on test data: 75.2%


In [ ]:
evaluate_experiment(y_val, multilabel_model.predict(intermediate_test_output))

,macro_auc
0,0.854471


In [ ]:
evaluate_experiment(y_train, multilabel_model.predict(intermediate_output))

,macro_auc
0,0.925169


In [ ]:
thresholds =  find_optimal_cutoff_thresholds(y_val , multilabel_model.predict(intermediate_test_output))

In [ ]:
y_pred_binary = apply_thresholds(multilabel_model.predict(intermediate_test_output), thresholds)
y_pred_binary[0]

array([0, 1, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
from sklearn.metrics import (confusion_matrix,
                             precision_score, recall_score, f1_score,
                             precision_recall_curve, average_precision_score , balanced_accuracy_score ,roc_auc_score)
diagnosis = ['NORM', 'AFIB', '1AVB', 'CLBBB', 'CRBBB', 'PAC', 'VPC', 'STD_', 'STE_']

def specificity_score(y_true, y_pred):
    m = confusion_matrix(y_true, y_pred, labels=[0, 1])
    spc = m[0, 0] * 1.0 / (m[0, 0] + m[0, 1])
    return spc

def get_scores(y_true, y_pred, score_fun):
    nclasses = np.shape(y_true)[1]
    scores = []
    for name, fun in score_fun.items():
        scores += [[fun(y_true[:, k], y_pred[:, k]) for k in range(nclasses)]]
    return np.array(scores).T


score_fun = {'Precision': precision_score,
             'Recall': recall_score, 'Specificity': specificity_score,
             'F1 score': f1_score , 'balanced_accuracy_score': balanced_accuracy_score}

scores_list = []
for y_pred in [y_pred_binary]:
    # Compute scores
    scores = get_scores(y_val, y_pred, score_fun)
    # Put them into a data frame
    scores_df = pd.DataFrame(scores, index=diagnosis, columns=score_fun.keys())
    # Append
    scores_list.append(scores_df)
# Concatenate dataframes
scores_all_df = pd.concat(scores_list, axis=1, keys=[''])
# Change multiindex levels
scores_all_df = scores_all_df.swaplevel(0, 1, axis=1)
scores_all_df = scores_all_df.reindex(level=0, columns=score_fun.keys())
scores_all_df

,Precision,Recall,Specificity,F1 score,balanced_accuracy_score
,,,,,
NORM,0.615385,0.888889,0.935275,0.727273,0.912082
AFIB,0.862903,0.869919,0.970018,0.866397,0.919968
1AVB,0.900000,0.750000,0.996997,0.818182,0.873498
CLBBB,0.952381,0.829016,0.983903,0.886427,0.906459
CRBBB,0.788889,0.771739,0.968227,0.780220,0.869983
PAC,0.666667,0.491803,0.976153,0.566038,0.733978
VPC,0.814815,0.758621,0.975124,0.785714,0.866873
STD_,0.769231,0.454545,0.995509,0.571429,0.725027
STE_,0.844444,0.542857,0.988710,0.660870,0.765783


In [ ]:
!pip install catboost

     |████████████████████████████████| 69.2 MB 4.9 kB/s 


In [ ]:
from catboost import CatBoostClassifier

cat_estimator = CatBoostClassifier(
                # iterations=10, 
                learning_rate=0.01, depth=10 , n_estimators=100 ,
                )

multilabel_model = MultiOutputClassifier(cat_estimator)


multilabel_model.fit(intermediate_output, y_train,  
                      # cat_features=cat_features, 
                      # eval_set=(intermediate_test_output, y_val),
                      # verbose=True
)

# print('CatBoost model is fitted: ' + str(multilabel_model.is_fitted()))
print('CatBoost model parameters:')
print(multilabel_model.get_params())

NameError: ignored

In [ ]:

evaluate_experiment(y_val, multilabel_model.predict(intermediate_test_output))

,macro_auc
0,0.700877


In [ ]:
evaluate_experiment(y_train, multilabel_model.predict(intermediate_output))

,macro_auc
0,0.913026


#sampling frequency=500


In [ ]:
from utils import utils

sampling_frequency=500
datafolder= '/gdrive/My Drive/ICBEB/'
task='all'
outputfolder='../output0/'

# Load data
data, raw_labels = utils.load_dataset(datafolder, sampling_frequency)
# Preprocess label data
labels = utils.compute_label_aggregations(raw_labels, datafolder, task)
# Select relevant data and convert to one-hot
data, labels, Y, _ = utils.select_data(data, labels, task, min_samples=0, outputfolder=outputfolder)

# 1-9 for training 
X_train = data[labels.strat_fold < 10]
y_train = Y[labels.strat_fold < 10]
# 10 for validation
X_val = data[labels.strat_fold == 10]
y_val = Y[labels.strat_fold == 10]

num_classes = 9         # <=== number of classes in the finetuning dataset
input_shape = [1000,12] # <=== shape of samples, [None, 12] in case of different lengths

X_train.shape, y_train.shape, X_val.shape, y_val.shape

((6187,), (6187, 9), (690,), (690, 9))

In [ ]:
data.shape
data = []

In [ ]:
min = X_train[0].shape[0]
for i in range(len(X_train)):
    if X_train[i].shape[0] < min:
        min = X_train[i].shape[0]
min


3000

In [ ]:
p = X_train[0][:,0]
p.shape[0]

7500

In [ ]:
X_tr = []
for i in range(len(X_train)):
    x = []
    for j in range(12):
        p = X_train[i][:,j]
        # print(p.shape[0])
        if p.shape[0] < 4000:
            d = abs(p.shape[0]-4000)//2
            p = np.pad(p,(d,d))
            # print(p.shape[0])
            if p.shape[0] != 4000:
                p = np.pad(p,(1,0))
                # print(p.shape)
        else:
            p = X_train[i][:4000,j]
        x.append(p)
    X_tr.append(np.transpose(x))
X_tr = np.array(X_tr)
X_tr.shape

(6187, 4000, 12)

In [ ]:
X_train = []

In [ ]:
X_te = []
for i in range(len(X_val)):
    x = []
    for j in range(12):
        p = X_val[i][:,j]
        # print(p.shape[0])
        if p.shape[0] < 4000:
            d = abs(p.shape[0]-4000)//2
            p = np.pad(p,(d,d))
            # print(p.shape[0])
            if p.shape[0] != 4000:
                p = np.pad(p,(1,0))
                print(p.shape)
        else:
            p = X_val[i][:4000,j]
        x.append(p)
    X_te.append(np.transpose(x))
X_te = np.array(X_te)
X_te.shape

(690, 4000, 12)

In [ ]:
X_train = []
X_val = []
for i in range(len(X_tr)):
    x = (X_tr[i] - np.mean(X_tr[i]))/(np.std(X_tr[i])+1e-30)
    X_train.append(x)
for i in range(len(X_te)):
    x = (X_te[i] - np.mean(X_te[i]))/(np.std(X_te[i])+1e-30)
    X_val.append(x)


In [ ]:
X_tr = []

In [ ]:
X_train = np.array(X_train)
X_val = np.array(X_val)

In [ ]:
main_input = Input(shape=(4000,12), dtype='float32', name='main_input')
x = Convolution1D(12, 3, padding='same')(main_input)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 3, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 24, strides = 2, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Dropout(0.2)(x)
x = Convolution1D(12, 3, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 3, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 24, strides = 2, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Dropout(0.2)(x)
x = Convolution1D(12, 3, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 3, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 24, strides = 2, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Dropout(0.2)(x)
x = Convolution1D(12, 3, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 3, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 24, strides = 2, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Dropout(0.2)(x)
x = Convolution1D(12, 3, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 3, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 48, strides = 2, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
cnnout = Dropout(0.2)(x)
x = Bidirectional(CuDNNGRU(12, input_shape=(125,12),return_sequences=True,return_state=False))(cnnout)
x = LeakyReLU(alpha=0.3)(x)
x = Dropout(0.2)(x)
x = AttentionWithContext()(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.3)(x)
x = Dropout(0.2)(x)
main_output = Dense(num_classes,activation='sigmoid')(x)

In [ ]:
model = Model(main_input,main_output)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      [(None, 4000, 12)]        0         
_________________________________________________________________
conv1d (Conv1D)              (None, 4000, 12)          444       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 4000, 12)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 4000, 12)          444       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 4000, 12)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 2000, 12)          3468      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 2000, 12)          0     

In [ ]:
model.compile(optimizer=Adam(learning_rate=1e-4),
                loss='binary_crossentropy',
                metrics=['AUC'])

In [ ]:
model.fit(X_train, y_train, 
          validation_split=0.1,
          epochs=20,
          batch_size=64,
          shuffle=True)

Epoch 1/20
87/87 [==============================] - 4s 47ms/step - loss: 0.1083 - auc: 0.9762 - val_loss: 0.1266 - val_auc: 0.9645
Epoch 2/20
87/87 [==============================] - 4s 43ms/step - loss: 0.1093 - auc: 0.9750 - val_loss: 0.1264 - val_auc: 0.9641
Epoch 3/20
87/87 [==============================] - 4s 46ms/step - loss: 0.1093 - auc: 0.9758 - val_loss: 0.1278 - val_auc: 0.9650
Epoch 4/20
87/87 [==============================] - 4s 46ms/step - loss: 0.1103 - auc: 0.9751 - val_loss: 0.1258 - val_auc: 0.9644
Epoch 5/20
87/87 [==============================] - 4s 45ms/step - loss: 0.1075 - auc: 0.9771 - val_loss: 0.1281 - val_auc: 0.9634
Epoch 6/20
87/87 [==============================] - 4s 45ms/step - loss: 0.1075 - auc: 0.9762 - val_loss: 0.1270 - val_auc: 0.9643
Epoch 7/20
87/87 [==============================] - 4s 44ms/step - loss: 0.1091 - auc: 0.9748 - val_loss: 0.1264 - val_auc: 0.9648
Epoch 8/20
87/87 [==============================] - 4s 43ms/step - loss: 0.1087 - a

In [ ]:
y_pred = model.predict(X_val)
utils.evaluate_experiment(y_val, y_pred)

,macro_auc
0,0.952921


In [ ]:
y_pred = model.predict(X_te)
utils.evaluate_experiment(y_val, y_pred)

,macro_auc
0,0.95758


In [ ]:
from sklearn.metrics import classification_report, fbeta_score, roc_auc_score, roc_curve, roc_curve, auc

def find_optimal_cutoff_threshold(target, predicted):
    """ 
    Find the optimal probability cutoff point for a classification model related to event rate
    """
    fpr, tpr, threshold = roc_curve(target, predicted)
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = threshold[optimal_idx]
    return optimal_threshold

def find_optimal_cutoff_thresholds(y_true, y_pred):
	return [find_optimal_cutoff_threshold(y_true[:,i], y_pred[:,i]) for i in range(y_true.shape[1])]

thresholds =  find_optimal_cutoff_thresholds_for_Gbeta(y_val , y_pred)


  0%|          | 0/9 [00:00<?, ?it/s]

optimize thresholds with respect to G_beta


100%|██████████| 9/9 [00:07<00:00,  1.14it/s]


In [ ]:
y_pred_binary = apply_thresholds(y_pred, thresholds)
y_pred_binary[0]

array([0, 1, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
utils.evaluate_experiment(y_val, y_pred_binary)

,macro_auc
0,0.905897


In [ ]:
from sklearn.metrics import (confusion_matrix,
                             precision_score, recall_score, f1_score,
                             precision_recall_curve, average_precision_score , balanced_accuracy_score)
diagnosis = ['NORM', 'AFIB', '1AVB', 'CLBBB', 'CRBBB', 'PAC', 'VPC', 'STD_', 'STE_']
def specificity_score(y_true, y_pred):
    m = confusion_matrix(y_true, y_pred, labels=[0, 1])
    spc = m[0, 0] * 1.0 / (m[0, 0] + m[0, 1])
    return spc

def get_scores(y_true, y_pred, score_fun):
    nclasses = np.shape(y_true)[1]
    scores = []
    for name, fun in score_fun.items():
        scores += [[fun(y_true[:, k], y_pred[:, k]) for k in range(nclasses)]]
    return np.array(scores).T


score_fun = {'Precision': precision_score,
             'Recall': recall_score, 'Specificity': specificity_score,
             'F1 score': f1_score , 'balanced accuracy': balanced_accuracy_score}

scores_list = []
for y_pred in [y_pred_binary]:
    # Compute scores
    scores = get_scores(y_val, y_pred, score_fun)
    # Put them into a data frame
    scores_df = pd.DataFrame(scores,index=diagnosis, columns=score_fun.keys())
    # Append
    scores_list.append(scores_df)
# Concatenate dataframes
scores_all_df = pd.concat(scores_list, axis=1, keys=[''])
# Change multiindex levels
scores_all_df = scores_all_df.swaplevel(0, 1, axis=1)
scores_all_df = scores_all_df.reindex(level=0, columns=score_fun.keys())

In [ ]:
#Gbeta thershold
scores_all_df

,Precision,Recall,Specificity,F1 score,balanced accuracy
,,,,,
NORM,0.873418,0.958333,0.983819,0.913907,0.971076
AFIB,0.879699,0.951220,0.971781,0.914062,0.961500
1AVB,0.846154,0.916667,0.993994,0.880000,0.955330
CLBBB,0.896714,0.989637,0.955734,0.940887,0.972686
CRBBB,0.783505,0.826087,0.964883,0.804233,0.895485
PAC,0.436893,0.737705,0.907790,0.548780,0.822748
VPC,0.740000,0.850575,0.956882,0.791444,0.903728
STD_,0.652174,0.681818,0.988024,0.666667,0.834921
STE_,0.842105,0.685714,0.985484,0.755906,0.835599


In [ ]:
# %% Confusion matrices (Supplementary Table 1)
from sklearn.metrics import (confusion_matrix,
                             precision_score, recall_score, f1_score,
                             precision_recall_curve, average_precision_score)

import xarray as xr

M = [[confusion_matrix(y_val[:, k], y_pred[:, k], labels=[0, 1])
      for k in range(9)] for y_pred in [y_pred_binary]]

M_xarray = xr.DataArray(np.array(M),
                        dims=['predictor', 'diagnosis', 'true label', 'predicted label'],
                        coords={'predictor': ['CNN+RNN'],
                                'diagnosis': diagnosis,
                                'true label': ['not present', 'present'],
                                'predicted label': ['not present', 'present']})
confusion_matrices = M_xarray.to_dataframe('n')
confusion_matrices = confusion_matrices.reorder_levels([1, 2, 3, 0], axis=0)
confusion_matrices = confusion_matrices.unstack()
confusion_matrices = confusion_matrices.unstack()
confusion_matrices = confusion_matrices['n']
confusion_matrices

predictor                 CNN+RNN        
predicted label       not present present
diagnosis true label                     
NORM      not present         608      10
          present               3      69
AFIB      not present         551      16
          present               6     117
1AVB      not present         662       4
          present               2      22
CLBBB     not present         475      22
          present               2     191
CRBBB     not present         577      21
          present              16      76
PAC       not present         571      58
          present              16      45
VPC       not present         577      26
          present              13      74
STD_      not present         660       8
          present               7      15
STE_      not present         611       9
          present              22      48